In [1]:
!pip install numpy
!pip install pandas

In [2]:
import numpy as np
import math

# Implementation of a Node
# A node should be able to take an input vector (which is either the input or the previous layer)
# Sum the values from the previous node
# Pass the sum to an activation value
# Return the value as its own value

class node(object):
  def __init__(self, inputVec = [], weight = [], activation = None):
    # Number of Inputs
    self.inputVector = inputVec
    self.nodeWeights = weight
    self.actSelect = activation
    self.bias = np.random.rand()

    # Summation of Connected Notes
    self.nodeSum = self.summationNodes()

    # Activation function
    self.nodeVal = self.activation()

  # Parallelizable for all input values
  def summationNodes(self):
    sumOfVec = 0
    for (i, j) in zip(self.inputVector, self.nodeWeights):
        sumOfVec = sumOfVec + i * j
    return (sumOfVec + self.bias)

  def activation(self):
    if (self.actSelect == "sigmoid"):
      return (1 / (1 + pow(math.e, -self.nodeSum)))

    if (self.actSelect == "relu"):
      return (max(0.1*self.nodeSum, self.nodeSum))

In [3]:
# For the mutli-layer perceptron
# We should first be able to store the input vector in an array
# Create N (user-defined number) nodes
# Allow users to add layers with custom nodes as a method of the class
# The output node automatically implements

class mlp(node):

  def __init__ (self):

    # We simulate the input layer here by storing each value
    # in the input as an element of a vector
    self.inputSize = None
    self.outputClass = []
    self.inputVector = [] # This is the vector we pass to the first hidden layer
    self.outputVal = None
    self.outputIndex = None

  def activate(self, inputVector, numClass):

    # We simulate the input layer here by storing each value
    # in the input as an element of a vector
    self.inputSize = len(inputVector)
    self.outputClass = numClass
    self.inputVector = inputVector # This is the vector we pass to the first hidden layer

    # Simulate hidden later
    layer1 = self.addLayer(numNodes = 10, prevLayer = self.inputVector)
    layer2 = self.addLayer(numNodes = 10, prevLayer = layer1)
    layer3 = self.addLayer(numNodes = 10, prevLayer = layer2)
    layer4 = self.addLayer(numNodes = 5, prevLayer = layer3)
    self.outputVal, self.outputIndex = self.outputLayer(layer4)

  # This intendeds to simulate the hidden layers
  def addLayer(self, numNodes = None, prevLayer = None):
    #weights = [np.random.rand() for i in range(len(inputVector))]

    if numNodes == None:
      numNodes = self.inputSize

    nextLayer = []
    for i in range(numNodes):
      # Create the nodes
      nextLayer.append(node(self.inputVector, [np.random.rand() for j in range(len(self.inputVector))], activation='relu'))

    outputValues = []
    for i in nextLayer:
      i.summationNodes()
      i.activation()
      outputValues.append(i.nodeVal)

    return outputValues

  def outputLayer(self, lastHidden):
    outLayer = []
    for i in range(self.outputClass):
      outLayer.append(node(lastHidden, [np.random.rand() for j in range(len(self.inputVector))], activation='sigmoid'))

    endVals = []
    for i in range(self.outputClass):
      endVals.append(outLayer[i].nodeVal)
      #print("Class ", i, " : ", outLayer[i].nodeVal)

    return max(endVals), endVals.index(max(endVals))

  def printResult(self):
    print("Predicted Class: ", self. outputIndex)

  def getPrediction(self):
    return self.outputIndex

# Sample with the Iris Dataset

In [4]:
# Loading the IRIS Dataset for Classification
!pip install ucimlrepo
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo

# fetch dataset
iris = fetch_ucirepo(id=53)

# data (as pandas dataframes)
df_X = iris.data.features
df_y = iris.data.targets

df_X = pd.DataFrame(df_X)
#df_X.head()

df_y = pd.DataFrame(df_y)
#df_y.head()

In [5]:
# Perform some data wrangling/preparation

# Check info on the attributes
df_X.info() # All values are same type, check if any missing values
df_X.isnull().sum() # To check if there are any null values

# Checking the output
df_y.info()

# Object Types must be converted
df_y_original = df_y.copy() # Store the df somewhere before modifying

df_y.value_counts()

# Reveals Three Classes
# Map classes to corresponding numerical values

outMap = {'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2}
df_y.replace(outMap, inplace=True)

print(df_y)
df_y.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal length  150 non-null    float64
 1   sepal width   150 non-null    float64
 2   petal length  150 non-null    float64
 3   petal width   150 non-null    float64
dtypes: float64(4)
memory usage: 4.8 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   150 non-null    object
dtypes: object(1)
memory usage: 1.3+ KB
     class
0        0
1        0
2        0
3        0
4        0
..     ...
145      2
146      2
147      2
148      2
149      2

[150 rows x 1 columns]


class
0        50
1        50
2        50
Name: count, dtype: int64

# Test out the CPU Model

In [6]:
def testModel(model, inVector, appendList = []):
  for i in inVector:
    model.activate(i, 3)
    appendList.append(model.getPrediction())

In [7]:
predictedVals = []
model = mlp()

testModel(model, np.array(df_X), predictedVals)

In [8]:
trueOutput = np.array(df_y)

correctPreds, totalPreds = 0, len(trueOutput)
for i in range(totalPreds):
  if predictedVals[i] == trueOutput[i]:
    correctPreds = correctPreds + 1

print("Correct Predictions / Total Predictions: {} / {} .".format(correctPreds, totalPreds))
print("Custom Feedforward Neural Network is: {} % Accurate".format(round(correctPreds/totalPreds * 100), 2))

Correct Predictions / Total Predictions: 51 / 150 .
Custom Feedforward Neural Network is: 34 % Accurate


# Implementation using CuPy

In [9]:
# For implementation with CuPY
import cupy as cp

In [14]:
# Reduction Kernel per Node
nodeSum_kernel = cp.ReductionKernel(
    'T inputVec, U inputWeight',  # input params
    'T outputVec',  # output params
    'inputVec * inputWeight',  # map
    'a + b',  # reduce
    'outputVec = a',  # post-reduction map
    '0',  # identity value
    'nodeSum_kernel'  # kernel name
)

class MLPcupy():

  def __init__ (self):
    # We simulate the input layer here by storing each value
    # in the input as an element of a vector
    self.inputSize = None
    self.outputClass = []
    self.inputVector = [] # This is the vector we pass to the first hidden layer
    self.outputVal = None
    self.outputIndex = None

  def activate(self, inputVector, numClass):
  # For now, this is just forward propagation

    # We simulate the input layer here by storing each value
    # in the input as an element of a vector
    self.inputSize = len(inputVector)
    self.outputClass = numClass
    self.inputVector = inputVector # This is the vector we pass to the first hidden layer

    # Simulate hidden later
    layer1 = self.addLayer(numNodes = 10, prevLayer = self.inputVector, activation = "relu")
    #print(layer1)
    layer2 = self.addLayer(numNodes = 10, prevLayer = layer1, activation = "relu")
    #print(layer2)
    layer3 = self.addLayer(numNodes = 10, prevLayer = layer2, activation = "relu")
    #print(layer3)
    layer4 = self.addLayer(numNodes = 5, prevLayer = layer3, activation = "relu")
    #print(layer4)

    # Simulate output layer
    self.outputIndex = self.outputLayer(layer4)

  # This intendeds to simulate the hidden layers
  def addLayer(self, numNodes = None, prevLayer = None, activation = ""):

    if numNodes == None:
      print("Indicate number of Nodes.")
      return

    # This will be where we store the new values
    newLayer = cp.zeros(numNodes)
    nodeValue = cp.zeros(len(prevLayer))

    for i in range(numNodes):
    # add_kernel(grid_size, block_size, (x1, x2, y, size))
    # nodeSum_kernel(const float* inputVector, const float* layerWeights, float* outputNode, const float* bias)
      #print("Nodes: ", prevLayer)
      weights = cp.array([cp.random.random() for i in range(len(prevLayer))])
      #print("Weights: ", weights)
      bias = cp.random.random()
      #print("Bias: ", bias)

      newLayer[i] = nodeSum_kernel(cp.array(prevLayer), cp.array(weights), cp.random.random()) + bias
      #print("Node ", i, " : ", newLayer[i])

      if (activation == "sigmoid"):
        newLayer[i] = (1 / (1 + pow(math.e, -1 * newLayer[i])))

      elif (activation == "relu"):
        newLayer[i] = (max(0.1*newLayer[i], newLayer[i]))

      else:
        print("ERROR: Invalid Choice.")
        return 0

    return newLayer

  def outputLayer(self, lastHidden):
    outLayer = []
    for i in range(self.outputClass):
      outLayer.append(node(lastHidden, [cp.random.random() for j in range(len(self.inputVector))], activation='sigmoid'))

    endVals = []
    for i in range(self.outputClass):
      endVals.append(outLayer[i].nodeVal)
      #print("Class ", i, " : ", outLayer[i].nodeVal)

    return endVals.index(max(endVals))

  def printResult(self):
    print("Predicted Class: ", self. outputIndex)

  def getPrediction(self):
    return self.outputIndex

## Testing CuPy Model

In [15]:
model2 = MLPcupy()
outputCuPy = cp.zeros(len(df_X))

In [16]:
def testGPUModel(model, inVector, outClasses, appendList = []):
  for i in range(len(inVector)):
    model.activate(cp.array(inVector[i]), outClasses)
    appendList[i] = model.getPrediction()

In [17]:
testGPUModel(model2, cp.array(df_X), 3, outputCuPy)

TypeError: __call__() takes exactly 3 positional arguments (2 given)

In [ ]:
trueOutput = np.array(df_y)
predictedCuPy = np.array(outputCuPy.get())

correctPreds, totalPreds = 0, len(trueOutput)
for i in range(totalPreds):
  if predictedCuPy[i] == trueOutput[i]:
    correctPreds = correctPreds + 1

print("Correct Predictions / Total Predictions: {} / {} .".format(correctPreds, totalPreds))
print("Custom Feedforward Neural Network is: {} % Accurate".format(round(correctPreds/totalPreds * 100), 2))